In [13]:
APP_URL='https://adk-default-service-name-641081974165.us-central1.run.app'

In [ ]:
## For application code: 
# import google.auth.transport.requests
# import google.oauth2.id_token
# request = google.auth.transport.requests.Request()
# id_token = google.oauth2.id_token.fetch_id_token(request, APP_URL)

In [75]:
# For local dev
import google.auth
import google.auth.transport.requests
import google.oauth2.id_token
import os

def get_id_token_local_env(cloud_run_url: str) -> str:
    """
    Obtains an ID token for a Cloud Run service in a local environment using ADC.
    Requires GOOGLE_APPLICATION_CREDENTIALS to be set.
    """
    credentials, project = google.auth.default(
        scopes=["https://www.googleapis.com/auth/cloud-platform"]
    )
    request = google.auth.transport.requests.Request()
    
    # If credentials don't have an ID token, refresh them to get one
    if not hasattr(credentials, 'id_token') or credentials.id_token is None:
        credentials.refresh(request)
        
    # If still no ID token, explicitly fetch one for the target audience
    if not hasattr(credentials, 'id_token') or credentials.id_token is None:
        id_token = google.oauth2.id_token.fetch_id_token(request, cloud_run_url, credentials=credentials)
    else:
        id_token = credentials.id_token

    return id_token

# Example usage:
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/path/to/your/service-account-key.json"
# cloud_run_service_url = "https://your-cloud-run-service-url.run.app"
# token = get_id_token_local_env(cloud_run_service_url)
# print(f"ID Token: {token}")

token = get_id_token_local_env(APP_URL)

In [76]:
APP_URL+'/list-apps'

'https://adk-default-service-name-641081974165.us-central1.run.app/list-apps'

In [77]:
# Authenticate 
import requests
# curl -X GET -H "Authorization: Bearer $TOKEN" $APP_URL/list-apps
headers = {
    "Authorization": f"Bearer {token}"
}

response = requests.get(APP_URL+'/list-apps', headers=headers)
response.text

'["test_agents"]'

In [86]:
# curl -X POST -H "Authorization: Bearer $TOKEN" \
#     $APP_URL/apps/capital_agent/users/user_123/sessions/session_abc \
#     -H "Content-Type: application/json" \
#     -d '{"state": {"preferred_language": "English", "visit_count": 5}}'
session_id = 'Sample COA non conform.pdf'
user_id = 'User_id'
session_headers = {
    "Authorization": f"Bearer {token}",
     "Content-Type": "application/json"
}
session_data = {
    "state": {
        "preferred_language": "English",
        "visit_count": 5
    }
}
session_url = APP_URL+f'/apps/test_agents/users/{user_id}/sessions/{session_id}'
response = requests.post(session_url, json=session_data, headers=session_headers)


In [87]:
response.text

'{"id":"Sample COA non conform.pdf","appName":"test_agents","userId":"User_id","state":{"state":{"preferred_language":"English","visit_count":5}},"events":[],"lastUpdateTime":1758230357.7113976}'

In [72]:
gcs_uri ='gs://schreiber-hackathon/Sample COA non conform.pdf'
prompt_url=APP_URL+'/run_sse'
prompt_headers = {
    "Authorization": f"Bearer {token}",
     "Content-Type": "application/json"
}

prompt_data ={
    "app_name": "test_agents",
    "user_id": f"{user_id}",
    "session_id":f"{session_id}",
    "new_message": {
        "role": "user",
        "parts": [{
        "text": gcs_uri
        }]
    },
    "streaming": False
    }
response = requests.post(prompt_url, json=prompt_data, headers=prompt_headers)


In [73]:
import json
# json.loads(response.text.split('data: ', 1)[1].strip())['content']['parts'][0]['text']
response.text

'data: {"content":{"parts":[{"thoughtSignature":"Co8HAR_MhbaKoayfg9-JIn-Bokn-H28m1TzMkh8GDstVrlMcEu7Of-v5fWCY9UkT7byeBb4m7Bbuasf3WiERVRltjxvVW1USyah2j8R_3-UxRxVsWjclUFOPTxiceozSV-ElgJiuPEKkC--4eXPHUfbgqZXzogV02eZqlA-TzNXG4RbWmC0soqHq7k1FVZPBvDtj0vbSxMDeQ-L4SdzwKdbjyCPjcX37VbiXkd2KI4_Kv7D_jyIWVj0wQX-HfYBJfSgzUv6lqo1CkgBv9iGML4iygy_c0Diy1RaEVC4SmdD-ZwMkotKQvR4ADwj9f2pKHg303b41AQwtvSV7Dtkz58l8GSG-kl8ceMwQBTIBCzQWqzP9tZJFLF6KdHxrtiVFLebBIHQc-81D1XfmXYnfTYU6XENJc_lEnsQKKulPnqwYur3CmLeFjfzjfVjS2I3nbWhcuPk5aXPaAsggSq6X1AocuOtNSRi1ypbk5PI7kxSoa0Qs0McC_FSD42JGzxa2xE4utbRSlrnQob0NZHrZ-0LCQWsty9JgSbHHvieMyFK6aOl7N313Glsaz2Oyvj9Dh-XhzyZGsBMdOfRDgc26wKY32BtZLKEJZWRaaJz30UpU_KXdT1r_QTFPuBtCBqD9BnlUJUOiHir3sP5vrPG3OSRGt7Xbaeduf0I9lSe6wtWDkc-mbW5qO0ndHcLxAXxiSWcM3Jw3ruvnPLBzoQ1ZOPh4uAWelUX1YF707eze8YoxT133KZepwarBEM4PqJDvULtQ4WZBpKtxCczpmAYHwYueVi_8IS1lb2fGlCZxotjNCRz5zVY4drmaAsqTNUwyo-xCBHI17q4PR8ixWQiTpwMfxy5aYuCaVT1Z5r5b0uJ090T1TqhJ9n9hYVpdBvoK5GT2C1PwE_lbuzognAlHoHaZEaTB9RwuYu2KvefUeKIEF6EH_ofxmB7

In [74]:
print(response.text.split('data: ', 1)[1])

{"content":{"parts":[{"thoughtSignature":"Co8HAR_MhbaKoayfg9-JIn-Bokn-H28m1TzMkh8GDstVrlMcEu7Of-v5fWCY9UkT7byeBb4m7Bbuasf3WiERVRltjxvVW1USyah2j8R_3-UxRxVsWjclUFOPTxiceozSV-ElgJiuPEKkC--4eXPHUfbgqZXzogV02eZqlA-TzNXG4RbWmC0soqHq7k1FVZPBvDtj0vbSxMDeQ-L4SdzwKdbjyCPjcX37VbiXkd2KI4_Kv7D_jyIWVj0wQX-HfYBJfSgzUv6lqo1CkgBv9iGML4iygy_c0Diy1RaEVC4SmdD-ZwMkotKQvR4ADwj9f2pKHg303b41AQwtvSV7Dtkz58l8GSG-kl8ceMwQBTIBCzQWqzP9tZJFLF6KdHxrtiVFLebBIHQc-81D1XfmXYnfTYU6XENJc_lEnsQKKulPnqwYur3CmLeFjfzjfVjS2I3nbWhcuPk5aXPaAsggSq6X1AocuOtNSRi1ypbk5PI7kxSoa0Qs0McC_FSD42JGzxa2xE4utbRSlrnQob0NZHrZ-0LCQWsty9JgSbHHvieMyFK6aOl7N313Glsaz2Oyvj9Dh-XhzyZGsBMdOfRDgc26wKY32BtZLKEJZWRaaJz30UpU_KXdT1r_QTFPuBtCBqD9BnlUJUOiHir3sP5vrPG3OSRGt7Xbaeduf0I9lSe6wtWDkc-mbW5qO0ndHcLxAXxiSWcM3Jw3ruvnPLBzoQ1ZOPh4uAWelUX1YF707eze8YoxT133KZepwarBEM4PqJDvULtQ4WZBpKtxCczpmAYHwYueVi_8IS1lb2fGlCZxotjNCRz5zVY4drmaAsqTNUwyo-xCBHI17q4PR8ixWQiTpwMfxy5aYuCaVT1Z5r5b0uJ090T1TqhJ9n9hYVpdBvoK5GT2C1PwE_lbuzognAlHoHaZEaTB9RwuYu2KvefUeKIEF6EH_ofxmB7og-nph9